In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests, io, json
import random
import plotly.express as px

import geopandas as gpd
import shapely.geometry


In [2]:
from ev import *
from radius_map import *

In [3]:
# bring in EV Data

ev_raw = pd.read_csv('./app/data_files/ev_car_final.csv',index_col=0)

ev_raw,ev_range,ev_df = ev_data(ev_raw)

# US Zip Code and Lat / Long Dataset    
us_zip_lat_long_data = pd.read_csv('./app/data_files/us_zip_code_lat_long.csv'
                                   ,dtype={'ZIP': str,'LAT': float,'LNG': float})

In [ ]:
# # # Get distinct Brands
# brands = ev_df['Brand'].unique()
# brands

In [ ]:
# # ev_df.loc[(ev_df['Brand']== 'Ford')]
# brand_1 = 'Tesla'
# # brand_2 = 'Volkswagen'

In [ ]:
# # # Get Distinct models from brands
# models = ev_df.index[(ev_df['Brand']== brand_1)].unique()
# models

In [ ]:
## Car 1
# brand_1 = 'Ford'
car_1 = 'Mustang Mach-E SR AWD'

## Car 2
# brand_2 = 'Volkswagen'
car_2 = 'ID.4 GTX'

In [ ]:
## Set Distiance
range_1 = get_range(ev_range,car_1)
range_2 = get_range(ev_range,car_2)

In [ ]:
## Set Zip 
# zip_search = input("Zip: ")
zip = '32026'

In [ ]:
# # Plot Map
# radius_map(zip,us_zip_lat_long_data,range_1,range_2)

In [ ]:
ev_raw.info()

In [ ]:
# Subset columns to one needed for correlation recomendation system
ev_df = ev_raw.iloc[:,[1,3,4,5,15,16,17,18]]

# Convert cagetorical columns into numerical and transpose
ev_df = pd.get_dummies(ev_df,drop_first = True).T

# Get correlations of vehicles with other vehicles 
corr = ev_df.corr()

# Set the diagonal values to nan
np.fill_diagonal(corr.values, np.nan)

# Get top 10 vehicles similar to  root vehicle
top10 = np.argsort(-corr.values, axis=1)[:, :10]

# Get correlation as numpy array for capturing values
corr_values = corr.to_numpy()


# Empty list
ev_top = []


# Loop through the top 10 correlations and build data frame

for i in  range(len(top10)):
        for j in range(len(top10[i])):
#             print("Model:", corr.columns[i])
#             print("Comparison Model:",corr.columns[order_top10[i][j]])
#             print("Correlation:",u[i][order_top10[i][j]])
            
            model =  corr.columns[i]
            comparison_model =corr.columns[top10[i][j]]
            correlation = corr_values[i][top10[i][j]]
            
            ev_top.append([model,comparison_model,correlation])
            
corr_final = pd.DataFrame(ev_top, columns=["Model", "Comparison Model","Correlation"])

In [ ]:

similar_df = corr_final.loc[(df_test['Model']== 'Cybertruck Tri Motor')][["Comparison Model","Correlation"]]

In [ ]:
similar_df

In [4]:
corr_final = get_corr(ev_raw)

In [5]:
car_id = 'Cybertruck Tri Motor'

In [9]:
car_id = 'Mustang Mach-E SR RWD'

In [10]:
get_similar(corr_final,car_id)

,Comparison Model,Correlation
460,2 Standard Range Single Motor,0.999951
461,Born 150 kW - 62 kWh,0.999868
462,Born 170 kW - 62 kWh,0.999704
463,ID.3 Pro Performance,0.999699
464,Megane E-Tech EV60 220hp,0.999685
465,ID.3 Pro,0.999653
466,ID.4 GTX,0.999639
467,Enyaq iV RS,0.999639
468,Enyaq iV 60,0.999461
469,e-Niro 64 kWh,0.999374


In [11]:
ev_df

,Brand,Battery Size (kWh),Segment,Powertrian,BodyStyle,Seats,0 - 62 MPH (Sec),Top Speed (MPH),Range (Mi),Efficiency (Wh/mi),Euro
Model,,,,,,,,,,,
U5,Aiways,63.0,C,FWD,SUV,5,7.5,93*,210*,300*,"€35,993"
e-tron 50 quattro,Audi,64.7,E,AWD,SUV,5,6.8,118,260,365,"€69,100"
e-tron 55 quattro,Audi,86.5,E,AWD,SUV,5,5.7,124,250,380,"€79,445"
e-tron GT quattro,Audi,85.0,F,AWD,Sedan,4,4.1,152,205*,325*,"€99,800"
e-tron GT RS,Audi,85.0,F,AWD,Sedan,4,3.3,155,175*,340*,"€138,200"
...,...,...,...,...,...,...,...,...,...,...,...
ID.4 Pure,Volkswagen,52.0,C,RWD,SUV,5,10.9,99,175,295,"€36,950"
ID.4 Pure Performance,Volkswagen,52.0,C,RWD,SUV,5,9,99,175,295,"€38,450"
C40 Recharge,Volvo,75.0,C,AWD,SUV,5,4.7,112,210,355*,"€62,050"


In [ ]:
corr = ev_df.corr()

In [ ]:
order_top10 = np.argsort(-corr.values, axis=1)[:, :10]

In [ ]:
u = corr.to_numpy()

In [ ]:
ev_top = []

for i in  range(len(order_top10)):
        for j in range(len(order_top10[i])):
#             print("Model:", corr.columns[i])
#             print("Comparison Model:",corr.columns[order_top10[i][j]])
#             print("Correlation:",u[i][order_top10[i][j]])
            
            model =  corr.columns[i]
            comparison_model =corr.columns[order_top10[i][j]]
            correlation = u[i][order_top10[i][j]]
            
            ev_top.append([model,comparison_model,correlation])
            
df_test = pd.DataFrame(ev_top, columns=["Model", "Comparison Model","Correlation"])

In [ ]:
df_test

In [ ]:
ev_top = []

for i in  range(len(d_test)):
        for j in range(len(d_test[i])):
            
            model =  corr.columns[i]
            comparison_model =corr.columns[d_test[i][j]]
            correlation = u[i][d_test[i][j]]
            
            ev_top.append([model,comparison_model,correlation])
            
df_test = pd.DataFrame(ev_top, columns=["Model", "Comparison Model","Correlation"])

In [ ]:
df_test

In [ ]:
order_top10[2]

In [ ]:
g = order_top10[2]

In [ ]:
corr.columns[order_top10][2][2]

In [ ]:
corr.to_numpy()[2][2]

In [ ]:
for i in order_top10[1]:
    print(corr.columns[i])
    print(u[1][i])

In [ ]:
for j in order_top10[1]:
    print(corr.columns[1])
    
    for i in order_top10[1]:
        print(corr.columns[i])
        print(u[1][i])

In [ ]:
u = corr.to_numpy()

In [ ]:
d_test = order_top10[:5]

In [ ]:
order_top10[0][1]

In [ ]:
for i in order_top10[0]:
    print(corr.columns[i])
    print(u[0][i])

In [ ]:
d_test[0][1]

In [ ]:
d_test

In [ ]:
rows = []
for i in range(3):
     rows.append([i, i + 1])

In [ ]:
ev_top = []

for i in  range(len(d_test)):
        for j in range(len(d_test[i])):
            print("Model:", corr.columns[i])
            print("Comparison Model:",corr.columns[d_test[i][j]])
            print("Correlation:",u[i][d_test[i][j]])
            
            model =  corr.columns[i]
            comparison_model =corr.columns[d_test[i][j]]
            correlation = u[i][d_test[i][j]]
            
            ev_top.append([model,comparison_model,correlation])
            
df_test = pd.DataFrame(ev_top, columns=["Model", "Comparison Model","Correlation"])

In [ ]:
df_test

In [ ]:
for i in d_test:
        for j in d_test[i]:
            print(corr.columns[i])
#             print(u[i][d_test[i][j]])

In [ ]:
d_test

In [ ]:
len(d_test)

In [ ]:
len(d_test[1])

In [ ]:
corr.columns[1]

In [ ]:
h = similar[similar < 1].unstack().transpose().drop_duplicates()

In [ ]:
similar = ev_df.corr()
np.fill_diagonal(similar.values, np.nan)

In [ ]:
order_top10 = np.argsort(-similar.values, axis=1)[:, :10]

In [ ]:
order_top10

In [ ]:
result_top10 = pd.DataFrame(
    similar.columns[order_top10], 
    columns=['1st', '2nd','3rd','4th','5th','6th','7th','8th','9th','10th'],
    index=similar.index
)

In [ ]:
result_top10

In [ ]:
# similar.reset_index(inplace=True)
similar.rename_axis("index",  inplace=True)

In [ ]:
similar

In [ ]:
for x in result_top10.columns:
    result_top10[x+"_Val"] = similar.lookup(similar.index, result_top10[x])

In [ ]:
for x in result_top10.columns:
    print(x)

In [ ]:
similar = similar.set_index(df["index"]).drop(columns="index")

In [ ]:
similar.lookup(similar.index, result_top10['1st'])

In [ ]:
similar[similar < .999999]

In [ ]:
similar[similar < 1]

In [ ]:
h = similar[similar < 1].unstack().transpose().drop_duplicates()

In [ ]:
h

In [ ]:
h.to_frame()

In [ ]:
df = similar[similar < 1].unstack().transpose()
    

In [ ]:
df = df.to_frame()

In [ ]:
df.index.set_names(["Model", "Comparison Model"], inplace=True)

In [ ]:
df.loc["Taycan Turbo"]

In [ ]:
df

In [ ]:
df.info()

In [ ]:
.unstack().transpose()\
    .sort_values( ascending=False)\
    .drop_duplicates()

In [ ]:
# https://towardsdatascience.com/recommender-system-in-python-part-2-content-based-system-693a0e4bb306
def get_similar_movies(movie_title, n_ratings_filter=100, n_recommendations=5):
    similar = matrix.corrwith(matrix[movie_title])
    corr_similar = pd.DataFrame(similar, columns=['correlation'])
    corr_similar.dropna(inplace=True)
    
    orig = data.copy()
    
    corr_with_movie = pd.merge(
        left=corr_similar, 
        right=orig, 
        on='title')[['title', 'correlation', 'numRatings']].drop_duplicates().reset_index(drop=True)
    
    result = corr_with_movie[corr_with_movie['numRatings'] > n_ratings_filter].sort_values(by='correlation', ascending=False)
    
    return result.head(n_recommendations)

In [ ]:
# # formatted_table(comparison_table)
# display(get_comparison_table(ev_df,car_1,car_2))
# print("*** shaded values are etimates ***")

In [ ]:
# get_comparison_table(ev_df,car_1,car_2)

In [ ]:
# a = get_data_table(car_1,car_2)

In [ ]:
# (a[car_1][1]-a[car_2][1])

In [ ]:
# diff = []

# for i in range(0,11):
    
#     try:
#         b = (int(a[car_1][i])) - (int(a[car_2][i]))
#     except:
#         try:
#             b = round((float(a[car_1][i].replace("*",""))) - (float(a[car_2][i].replace("*",""))),2)
#         except:
#             b = "NA"
#     diff.append(b)

In [ ]:
# diff

In [ ]:
# (float(a[car_1][1])) - (float(a[car_2][1]))

In [ ]:
# a[car_1]

In [ ]:
# (float(a[car_1][1].replace("*",""))) - (float(a[car_2][1].replace("*","")))

In [ ]:
# (int(a[car_1][2])) - (int(a[car_2][2]))
# (float(a[car_1][2].replace("*",""))) - (float(a[car_2][2].replace("*","")))